In [ ]:
import numpy as np
# import matplotlib.pyplot as plt
import rasterio
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error, accuracy_score, r2_score
from osgeo import gdal
from rasterio.transform import from_origin
from rasterio.mask import mask
import geopandas as gpd
from datetime import datetime
import os
import cv2

In [ ]:
lst_path = r"H:\ADM-Kenya\Workshop\LST\LST_1km"  # Change this to the path of your folder

vi_path = r"H:\ADM-Kenya\Workshop\LST\VI_20m"  # Change this to the path of your folder

output_path = r"H:\ADM-Kenya\Workshop\LST\LST_20m"  # Change this to the path of your folder

LST_files = [file for file in os.listdir(lst_path) if file.endswith(".tif")]
VI_files = [file for file in os.listdir(vi_path) if file.endswith(".tif")]

In [ ]:
for v in range(0, len(LST_files)):
    # Extract the date from the first LST file
    first_lst_date_string = LST_files[v].split("_")[1].split(".")[0]  # Remove the file extension
    first_lst_date = datetime.strptime(first_lst_date_string, "%Y%m%d")

    closest_vi_file = None
    min_date_difference = float('inf')  # Initialize with a large value

    # Iterate through VI files
    for vi_file in VI_files:
        # Extract the date from the VI file
        date_string_with_extension = vi_file.split("_")[2].split(".")[0]  # Remove the file extension
        vi_date = datetime.strptime(date_string_with_extension, "%Y-%m-%d")

        # Find the difference between the first LST date and the current VI date
        date_difference = abs(first_lst_date - vi_date).days

        # Check if the difference is smaller than the current minimum
        if date_difference <= min_date_difference:
            closest_vi_file = vi_file
            min_date_difference = date_difference

    # Check if the closest VI file has a difference less than or equal to 5 days
    if min_date_difference <= 5:
        print(f"First LST file {LST_files[v]} - OK. Closest VI file: {closest_vi_file}. Difference: {min_date_difference} days.")
        
        LST_1km = rasterio.open(fr'{lst_path}\\{LST_files[v]}')
        VI_20m = rasterio.open(fr'{vi_path}\\{closest_vi_file}')
        
        dataset = gdal.Open(fr'{lst_path}\\{LST_files[v]}')
        bands_lst = dataset.GetRasterBand(1)

        print('Number of Bands = ', dataset.RasterCount, '\nNumber of Horizontal Pixels = ', dataset.RasterXSize,  
              '\nNumber of Vertical Pixels = ', dataset.RasterYSize, '\nBands: ', bands_lst)
        
        lst_1km = LST_1km.read(1)
        lst_meta = LST_1km.read(1)
        # ndvi_1km = NDVI_1km.read(1)
        vi_20m = VI_20m.read()

        lst_1km_shape = lst_1km.shape
        num_bands_VI = vi_20m.shape[0]; print(num_bands_VI), vi_20m.shape


        new_width = lst_1km_shape[1]
        new_height = lst_1km_shape[0]
        filter_size = int(vi_20m.shape[2] / new_width)

        # Direct resampling while ignoring NaN values
        VI_20m_upscaled_full_2 = np.full((num_bands_VI, new_height, new_width), np.nan)

        for band in range(num_bands_VI): 
            for i in range(0, vi_20m.shape[1] - filter_size, filter_size):
                for j in range(0, vi_20m.shape[2] - filter_size, filter_size):
                    # Calculate the indices for the target array
                    target_i = i // filter_size
                    target_j = j // filter_size

                    if target_i < new_height and target_j < new_width:
                        block = vi_20m[band, i:i+filter_size, j:j+filter_size]
                        VI_20m_upscaled_full_2[band, target_i, target_j] = np.nanmean(block)



        VI_20m_upscaled_full_2.shape


        # Check for NaN values in the slice
        has_nan = np.isnan(VI_20m_upscaled_full_2[0,:,:])
  

        lst_1km = lst_1km.astype(float)
        lst_1km[lst_1km == -9999] = np.nan
        lst_1km[lst_1km < 273] = np.nan
        lst_1km.shape, np.nanmax(lst_1km), np.nanmean(lst_1km)


        nan_mask_lst = np.isnan(lst_1km)

        # Create a mask for NaN values in X_all_bands
        nan_mask_x_all = np.isnan(VI_20m_upscaled_full_2[0])

        # Combine the masks to create a final mask where both arrays have NaN values
        final_nan_mask = np.logical_or(nan_mask_lst, nan_mask_x_all)

        # Apply the final mask to both arrays
        lst_1km[final_nan_mask] = np.nan
        VI_20m_upscaled_full_2[:, final_nan_mask] = np.nan

        lst_1km.shape, VI_20m_upscaled_full_2.shape


        count_non_nan = np.count_nonzero(~np.isnan(VI_20m_upscaled_full_2[1,:,:]))


        # Flatten the 2D arrays to 1D while ignoring NaN values
        y = lst_1km[~final_nan_mask].flatten()

        # Flatten the 2D arrays to 1D while ignoring NaN values for X
        X_all_bands_flat = VI_20m_upscaled_full_2[:, ~final_nan_mask].T  # Transpose for proper shape

        # Create the regression model
        regression_model_rf = RandomForestRegressor(n_estimators=300, n_jobs=-1)

        # Fit the model
        regression_model_rf.fit(X_all_bands_flat, y)


        predictions_1km_rf = regression_model_rf.predict(X_all_bands_flat)
        residuals_rf = y - predictions_1km_rf

        residuals_rf.shape


        VI_20m_upscaled_full_2.shape, lst_1km.shape, np.nanmax(lst_1km)


        vi_1 = VI_20m.read(1)
        lst_dwscaled = cv2.resize(lst_1km, (vi_1.shape[1], vi_1.shape[0]), interpolation=cv2.INTER_NEAREST)


        nan_mask_lst_dws = np.isnan(lst_dwscaled)

        # Create a mask for NaN values in X_all_bands
        nan_mask_x_all_20m = np.isnan(vi_20m[0])

        # Combine the masks to create a final mask where both arrays have NaN values
        final_nan_mask_20m = np.logical_or(nan_mask_lst_dws, nan_mask_x_all_20m)

        # Apply the final mask to both arrays
        lst_dwscaled[final_nan_mask_20m] = np.nan
        vi_20m[:, final_nan_mask_20m] = np.nan


        vi_20m_flat = vi_20m[:, ~final_nan_mask_20m].T  # Transpose for proper shape

        downscaled_lst_20m = regression_model_rf.predict(vi_20m_flat) #X20m_valid_all_bands

        downscaled_lst_nan = np.empty_like(vi_1)

        # Fill the array with NaN values
        downscaled_lst_nan[:] = np.nan

        # Apply the mask to downscaled_lst_20m
        downscaled_lst_nan[~final_nan_mask_20m.reshape(vi_1.shape)] = downscaled_lst_20m


        # Create an array with NaN values of the same shape as ndvi_10m
        res_lst_nan = np.empty_like(lst_1km)
        res_lst_nan[:] = np.nan
        res_lst_nan[~final_nan_mask.reshape(lst_1km.shape)] = residuals_rf


        res_dwscaled = cv2.resize(res_lst_nan, (vi_1.shape[1], vi_1.shape[0]), interpolation=cv2.INTER_NEAREST)
        res_dwscaled.shape


        lst_c_20m = downscaled_lst_nan + res_dwscaled


        # Define the output file path

        output_file = fr'{output_path}\\{LST_files[v]}'

        # Get the metadata from the NDVI raster
        vi_metadata = VI_20m.meta

        # Update the metadata for the output GeoTIFF
        dst_metadata = {
            'driver': 'GTiff',
            'dtype': downscaled_lst_nan.dtype,
            'nodata': np.nan,
            'width': downscaled_lst_nan.shape[1],
            'height': downscaled_lst_nan.shape[0],
            'count': 1,  # Assuming a single-band GeoTIFF
            'crs': vi_metadata['crs'],
            'transform': vi_metadata['transform']
        }

        # Save the array as a GeoTIFF
        with rasterio.open(output_file, 'w', **dst_metadata) as dst:
            dst.write(lst_c_20m, 1)  # Assuming a single-band GeoTIFF
    else:
        print(f"First LST file {LST_files[v]} - No VI file found with a difference less than or equal to 5 days.")
        # Continue to the next LST file
        continue